# Download de dados
Esse notebook foi utilizado para possibilitar que todos os membros do grupo fizessem o download dos dados do banco Firebase.

## Funções Auxiliares
Import de biliotecas e definição de funções usadas para acessar o banco de dados e converter as informações obtidas em vetores numpy compatíveis com a biblioteca Skit-learn.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import firebase_admin
import json
import numpy as np
from firebase_admin import credentials, firestore

cred = credentials.Certificate("/content/serviceAccountKey.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [ ]:
def translate_y(y, inv=False):
  """
  O vetor de respostas sai do banco de dados como uma lista de caracteres,
  para possibilitar maior modularização na programação. Essa função traduz
  o vetor de respostas para um vetor de inteiros (para torná-la compatível
  com o modelo) e vice-versa (para traduzir a resposta do modelo).
  """
  result = []
  translate = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
    'i': 8,
    'j': 9,
    'k': 10,
    'l': 11,
    'm': 12,
    'n': 13,
    'o': 14,
    'p': 15,
    'q': 16,
    'r': 17,
    's': 18,
    't': 19,
    'u': 20,
    'v': 21,
    'w': 22,
    'x': 23,
    'y': 24,
    'z': 25,
  }
  translate_inv = {}
  for value in translate:
    translate_inv[str(translate[value])] = value
  
  if inv:
    return translate_inv[str(y)]

  for elem in y:
    if elem in translate:
      result.append(translate[elem])
    else:
      result.append(elem)
  return result


In [ ]:
def import_data():
  """
  Importa os dados do banco de dados e retorna um vetor de vetores de
  posicionamento das features das mãos e um vetor de respostas dizendo
  qual letra o sinal manual representa.
  """
  docs = db.collection("X").stream()

  y = []
  X = []
  for doc in docs:
    X_amostra = doc.to_dict()
    y_doc = db.collection("y").document(doc.id).get()
    if not y_doc.exists:
      continue
    y_amostra = y_doc.to_dict()

    y_amostra = y_amostra['value']
    X_amostra = X_amostra['value']

    y+=y_amostra
    X+=X_amostra

  y = translate_y(y)
  y = np.array(y).flatten()
  y = y.astype(int)
  X = np.array(X).reshape(-1, 63)

  return X, y

In [ ]:
def import_data_untranslated():
  """
  Similar a import_data, porém aqui a saída y é retornada
  como um array de strings. Útil para caso se queira trabalhar
  com um subset do alfabeto enviado para o banco (tal qual
  decidimos fazer no projeto).
  """
  docs = db.collection("X").stream()

  y = []
  X = []
  for doc in docs:
    X_amostra = doc.to_dict()
    y_doc = db.collection("y").document(doc.id).get()
    if not y_doc.exists:
      continue
    y_amostra = y_doc.to_dict()

    y_amostra = y_amostra['value']
    X_amostra = X_amostra['value']

    y+=y_amostra
    X+=X_amostra

  X = np.array(X).reshape(-1, 63)

  return X, y

## Baixa os dados
Baixa os dados completos do banco de dados.

In [ ]:
X, y = import_data()

## Testando com alfabetos diferentes
Aqui utilizamos a função de `import_data_unstranlated()` para testar com um subset de letras mais destacadas entre si. Por exemplo, como as letras "f" e "t" são extramamente similares, escolhemos manter apenas a letra mais popular no português - t - e descartar a outra. Dessa forma, pudemos garantir uma eficácia maior para o nosso modelo preditivo com o número limitado de dados que obtivemos.

In [ ]:
X_unt, y_unt = import_data_untranslated()

In [ ]:
def filter_alphabet(X_unt, y_unt, selected_alphabet):
  X = []
  y = []
  for i, letter in enumerate(y_unt):
    if letter in selected_alphabet:
      y.append(y_unt[i])
      X.append(X_unt[i])

  # converte y para sktlearn novamente
  y = translate_y(y)
  y = np.array(y).flatten()
  y = y.astype(int)

  return X, y

In [ ]:
selected_alphabet = ['a', 'o', 'e', 'r', 'n', 'd', 't', 'l', 'p', 'v', 'g', 'b', 'q']

X, y = filter_alphabet(X_unt, y_unt, selected_alphabet)

## Salva os dados em arquivos baixáveis
Salva os dados para serem baixados e acessados no notebook de teste de eficiência de modelos.

In [ ]:
import pickle

with open('X.data', 'wb') as f:
  pickle.dump(X, f)
with open('y.data', 'wb') as f:
  pickle.dump(y, f)